<hr>
<h3 style="color:green">設計一個 Python 程式，利用 cv2 工具模組，開啟兩個視窗，讀取一張圖片檔案，將此影像套用黃色濾鏡，顯示原始與過濾後的影像，按下鍵盤 Spacebar 關閉所有開啟的視窗。</h3>

In [1]:
import cv2
import numpy as np

# 開啟視窗
cv2.namedWindow('Flower')
cv2.namedWindow('Red')

# 讀取影像
img1 = cv2.imread('flower.jpg')

# 取得影像的尺寸，cv2 影像 shape 的意義與 Image 工具模組不同
height, width, depth = img1.shape

# 依照尺寸產生空白的 ndarray，預備存放處理後的影像
img2 = np.zeros([height,width,depth], dtype=np.uint8)

# 顯示尺寸確認
print(img1.shape)
print(img2.shape)

# 處理影像，注意：cv2 的色頻排列次序與 Image 工具模組不同，B-G-R
for row in range(height):
    for col in range(width):
        b = img1[row,col,0]
        g = img1[row,col,1]
        r = img1[row,col,2]
        img2[row,col,0] = 0
        img2[row,col,1] = 0
        img2[row,col,2] = r 

# 顯示影像
cv2.imshow('Flower', img1)
cv2.imshow('Red'   , img2)

# 等候鍵盤輸入 Spacebar 而結束
while (True):
    c = cv2.waitKey(500)
    if (c == ord(' ')):
        break

# 關閉所有顯示視窗
cv2.destroyAllWindows()

(480, 640, 3)
(480, 640, 3)


<hr>
<h3 style="color:green">設計一個 Python 程式，利用 cv2 工具模組，開啟視窗，讀取 mp4 視訊影片與自行設計的物件影像，去背之物件合成在影像畫面右上角，播放合成後的結果，播放完畢後關閉視窗。</h3>
<pre>
提示
注意所合成物件是否會超出原影像畫面，超出的部分會導致系統錯誤。
</pre>

In [1]:
import cv2
import numpy as np

# 開啟視窗
cv2.namedWindow('Mask')
cv2.namedWindow('invMask')

# 利用 cv2.VideoCapture 開啟視訊影片
cap = cv2.VideoCapture('yunlin.mov')

# 開啟合成物件影像
obj = cv2.imread('emoji.png')
h0, w0, _ = obj.shape

# 依據物件產生遮罩（Mask）影像
obj_gray = cv2.cvtColor(obj, cv2.COLOR_BGR2GRAY)
_ , mask = cv2.threshold(obj_gray, 240, 255, cv2.THRESH_BINARY_INV)
mask_inv = cv2.bitwise_not(mask)

# 檢查視訊檔案是否開啟成功
if (cap.isOpened() == False):
    print("開啟失敗！")

# 讀取視訊影片直到結束
while (cap.isOpened()):
    # 擷取每一個 Image Frame
    ret, frame = cap.read()
    # 如果畫面擷取成功，顯示所擷取的影像畫面
    if (ret == True):
        # 取得影像畫面的尺寸參數
        height, width, depth = frame.shape
        # 去背合成
        # 1. 取出目標區域影像原始畫面
        roi = frame[:h0,298:298+w0,:]
        # 2. 取出目標區的原始背景
        roi_bg = cv2.bitwise_and(roi, roi, mask = mask_inv)
        # 3. 取出目標區的物件前景
        obj_fg = cv2.bitwise_and(obj, obj, mask = mask)
        # 4. 目標區的背景與前景疊加合成
        dst = cv2.add(roi_bg, obj_fg)
        # 5. 合成結果更新為原始畫面目標區的新影像
        frame[:h0,298:298+w0,:] = dst
        # 顯示合成後的影像畫面
        cv2.imshow('Frame', frame)
        # 按下鍵盤 Space 立即結束播放（鍵盤的等候時間需要適度縮短）
        if (cv2.waitKey(25) == ord(' ')):
            break
    # 如果畫面擷取失敗，直接結束播放
    else:
        break

# 釋放視訊擷取所佔用的記憶體
cap.release()
 
# 關閉所有顯示視窗
cv2.destroyAllWindows()